In [ ]:
Các bước thực hiện:
Định nghĩa trạng thái: Một trạng thái của bài toán 8-puzzle được biểu diễn dưới dạng một ma trận 3x3.
Hàm đánh giá f(n) = g(n) + h(n):
g(n): Số bước từ trạng thái ban đầu đến trạng thái hiện tại.
h(n): Hàm heuristic (ước lượng chi phí từ trạng thái hiện tại đến trạng thái mục tiêu). Có thể sử dụng:
Số ô sai vị trí: Đếm số ô chưa đúng vị trí so với trạng thái đích.
Khoảng cách Manhattan: Tổng khoảng cách hàng và cột giữa các ô và vị trí mong muốn của chúng.
Tạo danh sách Open và Closed:
Open: Chứa các trạng thái chờ xử lý.
Closed: Chứa các trạng thái đã được xử lý.
Duyệt theo thuật toán A*
Chọn trạng thái có giá trị f(n) nhỏ nhất trong Open.
Sinh các trạng thái con bằng cách di chuyển ô trống (trái, phải, lên, xuống).
Nếu tìm thấy trạng thái đích → Kết thúc.
Nếu chưa tìm thấy → Tiếp tục mở rộng trạng thái tốt nhất.

Giải thích mã nguồn
Hàm manhattan_distance(state): Tính tổng khoảng cách Manhattan giữa các ô hiện tại và vị trí mục tiêu.
Lớp PuzzleNode: Đại diện cho một trạng thái của puzzle với thông tin state, g, h, và f.
Hàm get_neighbors(node): Sinh các trạng thái con bằng cách di chuyển ô trống.
Hàm reconstruct_path(node): Truy vết đường đi từ trạng thái cuối về trạng thái đầu.
Hàm a_star(initial_state): Triển khai thuật toán A* để tìm đường đi ngắn nhất.

In [1]:
import heapq
import numpy as np

# Trạng thái mục tiêu
GOAL_STATE = np.array([[1, 6, 2], [7, 3, 4], [8, 5, 0]])


def manhattan_distance(state):
  """Tính khoảng cách Manhattan"""
  distance = 0
  for i in range(3):
    for j in range(3):
      if state[i, j] != 0:
        goal_pos = np.argwhere(GOAL_STATE == state[i, j])[0]
        distance += abs(i - goal_pos[0]) + abs(j - goal_pos[1])
  return distance


class PuzzleNode:
  def __init__(self, state, parent=None, g=0):
    self.state = state
    self.parent = parent
    self.g = g
    self.h = manhattan_distance(state)
    self.f = self.g + self.h

  def __lt__(self, other):
    return self.f < other.f


def get_neighbors(node):
  """Sinh các trạng thái con"""
  neighbors = []
  x, y = np.argwhere(node.state == 0)[0]  # Vị trí ô trống
  moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]

  for dx, dy in moves:
    new_x, new_y = x + dx, y + dy
    if 0 <= new_x < 3 and 0 <= new_y < 3:
      new_state = node.state.copy()
      new_state[x, y], new_state[new_x, new_y] = new_state[new_x, new_y], new_state[x, y]
      neighbors.append(PuzzleNode(new_state, node, node.g + 1))

  return neighbors


def reconstruct_path(node):
  """Truy vết đường đi từ trạng thái cuối về trạng thái đầu"""
  path = []
  while node:
    path.append(node.state)
    node = node.parent
  return path[::-1]


def a_star(initial_state):
  """Thuật toán A*"""
  open_list = []
  heapq.heappush(open_list, PuzzleNode(initial_state))
  closed_set = set()

  while open_list:
    current_node = heapq.heappop(open_list)

    if np.array_equal(current_node.state, GOAL_STATE):
      return reconstruct_path(current_node)

    closed_set.add(tuple(current_node.state.flatten()))

    for neighbor in get_neighbors(current_node):
      if tuple(neighbor.state.flatten()) not in closed_set:
        heapq.heappush(open_list, neighbor)

  return None  # Không tìm thấy lời giải


# Trạng thái đầu vào (có thể thay đổi)
initial_state = np.array([[1, 3, 6], [8, 7, 2], [5, 0, 4]])
solution_path = a_star(initial_state)

if solution_path:
  print("Đường đi từ trạng thái ban đầu đến trạng thái mục tiêu:")
  for step, state in enumerate(solution_path):
    print(f"Bước {step}:")
    print(state)
    print()
else:
  print("Không tìm thấy lời giải")


Đường đi từ trạng thái ban đầu đến trạng thái mục tiêu:
Bước 0:
[[1 3 6]
 [8 7 2]
 [5 0 4]]

Bước 1:
[[1 3 6]
 [8 7 2]
 [0 5 4]]

Bước 2:
[[1 3 6]
 [0 7 2]
 [8 5 4]]

Bước 3:
[[1 3 6]
 [7 0 2]
 [8 5 4]]

Bước 4:
[[1 0 6]
 [7 3 2]
 [8 5 4]]

Bước 5:
[[1 6 0]
 [7 3 2]
 [8 5 4]]

Bước 6:
[[1 6 2]
 [7 3 0]
 [8 5 4]]

Bước 7:
[[1 6 2]
 [7 3 4]
 [8 5 0]]

